This is the notebook required to massage the data for the chord diagram found [here](https://rberenguel.github.io/sketches/chord/index.html).

The data in data.txt can be found in [SoccerStats](https://www.soccerstats.com/table.asp?league=spain_2020&tid=v), by selecting `Statistics` and then `Results Grid`. Copy that grid into a text file and tweak it a bit so the spacing and naming is uniform.

In [ ]:
import json
import numpy as np
import pandas as pd

data = pd.read_csv("data.txt")

In [ ]:
# Convert the strings defining the results into a 2-array per cell (that's a match, either home or away)

def transform(a):
    split = a.str.split('-')
    return split

transformed = data.apply(transform)

In [ ]:
# Compute the goal differential defined in each match array.

matrix = transformed.apply(lambda c: c.apply(lambda x: int(x[1])-int(x[0]) if len(x)>1 else 0))

In [ ]:
# Compute the overall (home-away) goal differential

reflexed = (-np.tril(matrix)).T+np.triu(matrix)
goal_diff_matrix = np.power(2.0, reflexed-reflexed.T)

In [ ]:
# Normalise the matrix (always a good plan) and convert to JSON. Store the labels separately.

chord = goal_diff_matrix
norm = np.linalg.norm(chord)

normalized = chord/norm

with open("matrix.json", "w") as f:
    json.dump(normalized.tolist(), f)
    
categories = transformed.columns.tolist()
    
pd.DataFrame(["name"] + categories).to_csv("categories.csv", index=False, header=False)